## Decomposed Evo Algo Architecture

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
hf_api_token = os.getenv("HF_API_TOKEN")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
import re

def get_json(response):
    # Retrieve json from response
    pattern = rf"```json\s+(.*?)\s+```"
    json_block = re.search(pattern, response.content, re.DOTALL) # Using re.DOTALL to make the dot match newlines as well
    json_code = None
    if json_block:
        extracted_json = json_block.group(1)
        json_code = extracted_json
    return json_code

### Stage 1: Prompt Pool Generation

An agent that generates an initial pool of domain specific prompts to initiate the algorithm.

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain_community.llms import HuggingFaceEndpoint
import json

In [5]:
class ThinkingStylesAgent:

    def __init__(self, n: int, base_prompt: str):
        self.n = n
        self.base_prompt = base_prompt

    def generate(self):
        """
        Agent that generates n different prompts with different thinking styles to approach a probelm.
        """

        # Load open source language model
        # model = "mistralai/Mistral-7B-Instruct-v0.3"
        # llm = HuggingFaceEndpoint(
        #     repo_id=model,
        #     huggingfacehub_api_token=hf_api_token
        # )

        # Load openai language model
        llm = ChatOpenAI(
            api_key=openai_api_key,
            temperature=0.5,
            model='gpt-4o'
        )

        # Agent instruction
        instruction = f"""
        You are an expert thinker and problem solver. Here is a base prompt: {self.base_prompt}
        Generate {self.n} prompts to elaborate on the base prompt, each with a unique thinking style. 
        Consider the context of the problem and the end user (an LLM) of the prompt.
        
        The output should be json code containing {self.n} prompts with the following format:
        {{
            "style 1": {{
                "prompt": "prompt content",
                "thinking style": "thinking style",
            }},
            "style 2": {{
                "prompt": "prompt content",
                "thinking style": "thinking style",
            }},
            ...
        }}
        """

        # Generate prompts
        model_output = llm.invoke(instruction)
        json_output = get_json(model_output)

        if json_output is None:
            json_output = model_output.content

        return json.loads(json_output)

In [6]:
base_prompt = "Write a function to calculate the factorial of a number."
thinking_styles_agent = ThinkingStylesAgent(10, base_prompt)
thinking_styles = thinking_styles_agent.generate()

/Users/iwatson/Documents/Research Project/prompt-optimisation/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
thinking_styles

{'style 1': {'prompt': 'Write a function in Python to calculate the factorial of a number using a recursive approach. Ensure the function handles edge cases like zero and negative numbers.',
  'thinking style': 'analytical'},
 'style 2': {'prompt': 'Create a Python function that computes the factorial of a given number. Use an iterative approach and explain why you chose this method over recursion.',
  'thinking style': 'comparative'},
 'style 3': {'prompt': 'Design a Python function to calculate the factorial of a number. Include comments in your code to explain each step and make it easy to understand for beginners.',
  'thinking style': 'pedagogical'},
 'style 4': {'prompt': 'Implement a Python function that calculates the factorial of a number. Consider optimizing the function for large numbers and discuss the trade-offs involved.',
  'thinking style': 'optimization-focused'},
 'style 5': {'prompt': 'Write a Python function to compute the factorial of a number. Include error handli

In [8]:
# from https://arxiv.org/pdf/2309.16797.pdf [pg. 21] 

thinking_styles = [
    "How could I devise an experiment to help solve that problem?",
    "Make a list of ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.",
    "How could I measure progress on this problem?",
    "How can I simplify the problem so that it is easier to solve?",
    "What are the key assumptions underlying this problem?",
    "What are the potential risks and drawbacks of each solution?",
    "What are the alternative perspectives or viewpoints on this problem?",
    "What are the long-term implications of this problem and its solutions?",
    "How can I break down this problem into smaller, more manageable parts?",
    "Critical Thinking: This style involves analyzing the problem from different perspectives, questioning assumptions, and evaluating the evidence or information available. It focuses on logical reasoning, evidence-based decision-making, and identifying potential biases or flaws in thinking.",
    "Try creative thinking, generate innovative and out-of-the-box ideas to solve the problem. Explore unconventional solutions, thinking beyond traditional boundaries, and encouraging imagination and originality.",
    "Seek input and collaboration from others to solve the problem. Emphasize teamwork, open communication, and leveraging the diverse perspectives and expertise of a group to come up with effective solutions.",
    "Use systems thinking: Consider the problem as part of a larger system and understanding the interconnectedness of various elements. Focuses on identifying the underlying causes, feedback loops, and interdependencies that influence the problem, and developing holistic solutions that address the system as a whole.",
    "Use Risk Analysis: Evaluate potential risks, uncertainties, and tradeoffs associated with different solutions or approaches to a problem. Emphasize assessing the potential consequences and likelihood of success or failure, and making informed decisions based on a balanced analysis of risks and benefits.",
    "Use Reflective Thinking: Step back from the problem, take the time for introspection and self-reflection. Examine personal biases, assumptions, and mental models that may influence problem-solving, and being open to learning from past experiences to improve future approaches.",
    "What is the core issue or problem that needs to be addressed?",
    "What are the underlying causes or factors contributing to the problem?",
    "Are there any potential solutions or strategies that have been tried before? If yes, what were the outcomes and lessons learned?",
    "What are the potential obstacles or challenges that might arise in solving this problem?",
    "Are there any relevant data or information that can provide insights into the problem? If yes, what data sources are available, and how can they be analyzed?",
    "Are there any stakeholders or individuals who are directly affected by the problem? What are their perspectives and needs?",
    "What resources (financial, human, technological, etc.) are needed to tackle the problem effectively?",
    "How can progress or success in solving the problem be measured or evaluated?",
    "What indicators or metrics can be used?",
    "Is the problem a technical or practical one that requires a specific expertise or skill set? Or is it more of a conceptual or theoretical problem?",
    "Does the problem involve a physical constraint, such as limited resources, infrastructure, or space?",
    "Is the problem related to human behavior, such as a social, cultural, or psychological issue?",
    "Does the problem involve decision-making or planning, where choices need to be made under uncertainty or with competing objectives?",
    "Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?",
    "Is the problem a design challenge that requires creative solutions and innovation?",
    "Does the problem require addressing systemic or structural issues rather than just individual instances?",
    "Is the problem time-sensitive or urgent, requiring immediate attention and action?",
    "What kinds of solution typically are produced for this kind of problem specification?",
    "Given the problem specification and the current best solution, have a guess about other possible solutions.",
    "Let's imagine the current best solution is totally wrong, what other ways are there to think about the problem specification?",
    "What is the best way to modify this current best solution, given what you know about these kinds of problem specification?",
    "Ignoring the current best solution, create an entirely new solution to the problem.",
    "Let's think step by step.",
    "Let's make a step by step plan and implement it with good notion and explanation."
]

In [9]:
# class PromptPoolGeneration:
#     """
#     Use a selection of thinking styles to generate a pool of prompts for a given problem.
#     """

#     def __init__(self, task: str, thinking_styles: dict, n: int):
#         self.task = task
#         self.thinking_styles = thinking_styles
#         self.n = n

#     def generate(self, previous_cycle_results: dict={}):
#         """
#         Generate n prompts based on the thinking styles.
#         """

#         # Load openai language model
#         llm = ChatOpenAI(
#             api_key=openai_api_key,
#             temperature=0.5,
#             model='gpt-4o'
#         )

#         # Agent instruction
#         instruction = f"""
#         You are an expert prompt engineer. Generate a pool of {self.n} prompts, based on a selection of the following thinking styles: {self.thinking_styles}. 
#         The prompts should direct an langauge model to perform the following task: {self.task}.
#         If avaliable, the pool of prompts must include the best mutated prompts from the following previous cycle results exactly as presented and use the feedback to inform the generation of new prompts: {previous_cycle_results}.
        
#         The output must be json code containing {self.n} prompts (including any mutated prompts from the previous cycle) with the following format: 
#         {{
#             "prompt 1": "prompt content",
#             "prompt 2": "prompt content",
#             ...
#         }}
#         """

#         # Generate prompts
#         model_output = llm.invoke(instruction)
#         json_output = get_json(model_output)

#         if json_output is None:
#             json_output = model_output.content

#         return json.loads(json_output)

In [10]:
# problem = "write a function to calculate the factorial of a number."

In [11]:
# prompt_pool_generation = PromptPoolGeneration(problem, thinking_styles, 3)
# prompts = prompt_pool_generation.generate()

In [12]:
# prompts

### Stage 2: Selective Breeding

Task an LLM with pairing prompts it believes will produce effective prompts as offspring (now merged with stage 3)

In [13]:
# class MatchMakerAgent:
#     """
#     Agent that pairs two prompts for breeding new ideas.
#     """

#     def __init__(self, prompts: dict, n: int):
#         self.prompts = prompts
#         self.n = n

#     def generate(self):
#         """
#         Generate pairs of prompts.
#         """

#         # Load openai language model
#         llm = ChatOpenAI(
#             api_key=openai_api_key,
#             temperature=0.5,
#             model='gpt-4o'
#         )

#         # Agent instruction
#         instruction = f"""
#         You are an expert thinker and problem solver. Generate {self.n} pairs of parent prompts from the following list of prompts: {self.prompts}.
#         Think carefully about how the prompts can be combined to create new ideas. Justify why you think the prompts are a good match. 
#         Reminder, the prompts should instruct a language model to perfrom the task at hand.

#         The output must be json code containing {self.n} pairs of prompts with the following format:
#         {{
#             "pair 1": {{
#                 "parent 1": "parent prompt 1",
#                 "parent 2": "parent prompt 2",
#                 "justification": "justification"
#             }},
#             "pair 2": {{
#                 "parent 1": "parent prompt 1",
#                 "parent 2": "parent prompt 2",
#                 "justification": "justification"
#             }},
#             ...
#         }}
#         """

#         # Generate pairs of prompts
#         model_output = llm.invoke(instruction)
#         json_output = get_json(model_output)

#         if json_output is None:
#             json_output = model_output.content

#         return json.loads(json_output)

In [14]:
# match_maker_agent = MatchMakerAgent(prompts, 1)
# pairs_json = match_maker_agent.generate()

In [15]:
# pairs_json

### Stage 3: Genetic Engineering

Breed prompts, combining the "best" parts of each prompt.

In [16]:
class BreederAgent:
    """
    Agent that pairs two prompts and breeds them to create new a prompt.
    """

    def __init__(self, prompts: dict, n: int):
        self.prompts = prompts
        self.n = n

    def generate(self):
        """
        Generate pairs of prompts.
        """

        # Load openai language model
        llm = ChatOpenAI(
            api_key=openai_api_key,
            temperature=0.5,
            model='gpt-4o'
        )

        # Agent instruction
        instruction = f"""
        You are an expert thinker and problem solver. Select the most effective {self.n} prompts from the following list of prompts: {self.prompts}.
        From your selection, generate {self.n/2} pairs of parent prompts. Think carefully about how the prompts can be combined to create new ideas. Justify why you think the prompts are a good match.
        Once you have generated the pairs, breed the prompts to create new prompts. Combine the prompts in a way that enhances the original ideas and creates new insights. 
        Reminder, the prompts should instruct a language model to perfrom the task at hand.

        The output must be json code containing {self.n/2} pairs of prompts with the following format:
        {{
            "pair 1": {{
                "parent 1": "parent prompt 1",
                "parent 2": "parent prompt 2",
                "child": "child prompt",
                "pairing justification": "justification"
            }},
            "pair 2": {{
                "parent 1": "parent prompt 1",
                "parent 2": "parent prompt 2",
                "child": "child prompt",
                "pairing justification": "justification"
            }},
            ...
        }}
        """

        # Generate pairs of prompts
        model_output = llm.invoke(instruction)
        json_output = get_json(model_output)

        if json_output is None:
            json_output = model_output.content

        return json.loads(json_output)

In [17]:
# class BreederAgent:
#     """
#     Breed new ideas from pairs of prompts taking inspiration from biological evolution.
#     """

#     def __init__(self, pairs: dict):
#         self.pairs = pairs
#         self.n = len(pairs)

#     def generate(self):
#         """
#         Generate n new ideas from pairs of prompts.
#         """

#         # Load openai language model
#         llm = ChatOpenAI(
#             api_key=openai_api_key,
#             temperature=0.5,
#             model='gpt-4o'
#         )

#         # Agent instruction
#         instruction = f"""
#         You are an expert thinker and problem solver. Generate new prompts by breeding the pairs of parent prompts defined here: {self.pairs}.
#         Reminder, the prompts should instruct a language model to perfrom the task at hand. 
#         Think carefully about how the prompts can be combined to produce children, considering the best aspects of each parent. 
#         Detail how you have combined the parent prompts and why you think the child prompts are innovative and useful. 

#         The output must be json code containing {self.n} child prompts with the following format:
#         {{
#             "child 1": {{
#                 "prompt": "child prompt",
#                 "details": "breeding details"
#             }},
#             "child 2": {{
#                 "prompt": "child prompt",
#                 "details": "breeding details"
#             }},
#             ...
#         }}
#         """

#         # Generate new ideas
#         model_output = llm.invoke(instruction)
#         json_output = get_json(model_output)

#         if json_output is None:
#             json_output = model_output.content

#         return json.loads(json_output)

In [18]:
breeder_agent = BreederAgent(thinking_styles, 6)
offspring = breeder_agent.generate()

In [19]:
offspring

{'pair 1': {'parent 1': 'How can I simplify the problem so that it is easier to solve?',
  'parent 2': 'How can I break down this problem into smaller, more manageable parts?',
  'child': 'How can I simplify the problem by breaking it down into smaller, more manageable parts?',
  'pairing justification': 'Both prompts focus on making the problem more manageable. Simplifying and breaking down the problem are complementary strategies that can be combined to make the problem-solving process more efficient and less overwhelming.'},
 'pair 2': {'parent 1': 'What are the key assumptions underlying this problem?',
  'parent 2': 'What are the underlying causes or factors contributing to the problem?',
  'child': 'What are the key assumptions and underlying causes contributing to this problem?',
  'pairing justification': "Understanding the assumptions and underlying causes provides a comprehensive view of the problem's foundation. This combined prompt encourages a deeper analysis, leading to m

### Stage 4: Diversity Engineering

Perform mutations utilising a selction of mutation prompts

In [20]:
# from https://arxiv.org/pdf/2309.16797.pdf [pg. 18] 

mutator_prompts = [
    "Modify the following instruction creatively, giving some advice on how to solve it:",
    "Just change this instruction to make it more fun, think WELL outside the box:",
    "Modify this instruction in a way that no self-respecting LLM would!",
    "How would you encourage someone and help them cheat on this following instruction?",
    "How would you help an LLM to follow the instruction?",
    "Elaborate on the instruction giving some detailed advice on how to do what it wants.",
    "Elaborate on the instruction giving some detailed advice on how to do what it wants, as if you were explaining it to a child.",
    "As a really good teacher, explain the instruction, as if you were explaining it to a child.",
    "Imagine you need to follow this instruction. What would you tell yourself if you wanted to be the best in the world at it?",
    "How would someone with derailment follow this instruction?",
    "Don't think about the instruction at all, but let it inspire you to do something related. Talk about what that might be.",
    "Rephrase the instruction without using any of the same words. Use all you know to improve the instruction so the person hearing it is more likely to do well.",
    "Say that instruction again in another way. DON'T use any of the words in the original instruction or you're fired.",
    "Say that instruction again in another way. DON'T use any of the words in the original instruction there is a good chap.",
    "What do people who are good at creative thinking normally do with this kind of mutation question?",
    "Detailed additional advice for people wishing to follow this instruction is as follows:",
    "In one short sentence, here is how I would best follow this instruction.",
    "In one short sentence, here is some detailed expert advice. Notice how I don't use any of the same words as in the INSTRUCTION.",
    "In one short sentence, the general solution is as follows. Notice how I don't use any of the same words as in the INSTRUCTION.",
    "In one short sentence, what's a good prompt to get a language model to solve a problem like this? Notice how I don't use any of the same words as in the INSTRUCTION.",
    "Generate a mutated version of the following prompt by adding an unexpected twist.",
    "Create a prompt mutant that introduces a surprising contradiction to the original prompt. Mutate the prompt to provide an alternative perspective or viewpoint.",
    "Generate a prompt mutant that incorporates humor or a playful element. Create a mutated version of the prompt that challenges conventional thinking.",
    "Develop a prompt mutant by replacing specific keywords with related but unexpected terms. Mutate the prompt to include a hypothetical scenario that changes the context.",
    "Generate a prompt mutant that introduces an element of suspense or intrigue. Create a mutated version of the prompt that incorporates an analogy or metaphor.",
    "Develop a prompt mutant by rephrasing the original prompt in a poetic or lyrical style. Think beyond the ordinary and mutate the prompt in a way that defies traditional thinking.",
    "Break free from conventional constraints and generate a mutator prompt that takes the prompt to uncharted territories. Challenge the norm and create a mutator prompt that pushes the boundaries of traditional interpretations.",
    "Embrace unconventional ideas and mutate the prompt in a way that surprises and inspires unique variations. Think outside the box and develop a mutator prompt that encourages unconventional approaches and fresh perspectives.",
    "Step into the realm of imagination and create a mutator prompt that transcends limitations and encourages innovative mutations. Break through the ordinary and think outside the box to generate a mutator prompt that unlocks new possibilities and unconventional paths.",
    "Embrace the power of unconventional thinking and create a mutator prompt that sparks unconventional mutations and imaginative outcomes. Challenge traditional assumptions and break the mold with a mutator prompt that encourages revolutionary and out-of-the-box variations.",
    "Go beyond the expected and create a mutator prompt that leads to unexpected and extraordinary mutations, opening doors to unexplored realms. Increase Specificity: If the original prompt is too general, like 'Tell me about X,' the modified version could be, 'Discuss the history, impact, and current status of X.'",
    "Ask for Opinions/Analysis: If the original prompt only asks for a fact, such as 'What is X?', the improved prompt could be, 'What is X, and what are its implications for Y?'",
    "Encourage Creativity: For creative writing prompts like 'Write a story about X,' an improved version could be, 'Write a fantasy story about X set in a world where Y is possible.'",
    "Include Multiple Perspectives: For a prompt like 'What is the impact of X on Y?', an improved version could be, 'What is the impact of X on Y from the perspective of A, B, and C?'",
    "Request More Detailed Responses: If the original prompt is 'Describe X,' the improved version could be, 'Describe X, focusing on its physical features, historical significance, and cultural relevance.'",
    "Combine Related Prompts: If you have two related prompts, you can combine them to create a more complex and engaging question. For instance, 'What is X?' and 'Why is Y important?' could be combined to form 'What is X and why is it important in the context of Y?'",
    "Break Down Complex Questions: If a prompt seems too complex, like 'Discuss X,' the improved version could be, 'What is X? What are its main characteristics? What effects does it have on Y and Z?'",
    "Use Open-Ended Questions: Instead of 'Is X true?', you could ask, 'What are the arguments for and against the truth of X?'",
    "Request Comparisons: Instead of 'Describe X,' ask 'Compare and contrast X and Y.'",
    "Include Context: If a prompt seems to lack context, like 'Describe X,' the improved version could be, 'Describe X in the context of its impact on Y during the Z period.'",
    "Make the prompt more visual: Ask the user to visualize the problem or scenario being presented in the prompt.",
    "Ask for a thorough review: Instead of just presenting the problem, ask the user to write down all the relevant information and identify what's missing.",
    "Invoke previous experiences: Modify the prompt to ask the user to recall a similar problem they've successfully solved before.",
    "Encourage a fresh perspective: Suggest in your prompt that the user take a moment to clear their mind before re-approaching the problem.",
    "Promote breaking down problems: Instead of asking the user to solve the problem as a whole, prompt them to break it down into smaller, more manageable parts.",
    "Ask for comprehension: Modify the prompt to ask the user to review and confirm their understanding of all aspects of the problem.",
    "Suggest explanation to others: Change the prompt to suggest that the user try to explain the problem to someone else as a way to simplify it.",
    "Prompt for solution visualization: Instead of just asking for the solution, encourage the user to imagine the solution and the steps required to get there in your prompt.",
    "Encourage reverse thinking: Improve the prompt by asking the user to think about the problem in reverse, starting with the solution and working backwards.",
    "Recommend taking a break: Modify the prompt to suggest that the user take a short break, allowing their subconscious to work on the problem.",
    "What errors are there in the solution?",
    "How could you improve the working out of the problem?",
    "Look carefully to see what you did wrong, how could you fix the problem?",
    "CORRECTION =",
    "Does the above text make sense? What seems wrong with it? Here is an attempt to fix it:",
    "The above working out has some errors, here is a version with the errors fixed."
]

In [37]:
class MutationAgent:
    """
    Perfrom mutations on the offspring to encourage diversity and generate unique ideas, taking inspiration from the concept of genetic mutation in biological evolution.
    Utilise the mutator prompts provided
    """

    def __init__(self, offspring: dict, mutator_prompts: list):
        self.offspring = offspring
        self.mutator_prompts = mutator_prompts

    def generate(self):
        """
        Generate n mutated ideas from offspring prompts.
        """

        # Load openai language model
        llm = ChatOpenAI(
            api_key=openai_api_key,
            temperature=0.5,
            model='gpt-4o'
        )

        # Agent instruction
        instruction = f"""
        You are an expert thinker and problem solver. Generate new prompts by mutating the following child prompts: {self.offspring}. 
        Perform a mutation on each child prompt. Tailor each mutation to each prompt, using the mutator prompt from the following selection you deem most applicable: {self.mutator_prompts}.
        Reminder, the prompts should instruct a language model to perfrom the task at hand. 
        Detail how the prompts have been mutated.

        The output must be json code with the following format:
        {{
            "child prompt 1": {{
                "original prompt": "original prompt",
                "mutated prompt": "mutated child prompt",
                "details": "mutation details"
                }},
            "child prompt 2": {{
                ...
            }},
            ...
        }}
        """

        # Generate new ideas
        model_output = llm.invoke(instruction)
        json_output = get_json(model_output)

        if json_output is None:
            json_output = model_output.content

        return json.loads(json_output)

In [38]:
mutation_agent = MutationAgent(offspring, mutator_prompts)
mutated_offspring = mutation_agent.generate()

In [39]:
mutated_offspring

{'child prompt 1': {'original prompt': 'How can I simplify the problem by breaking it down into smaller, more manageable parts?',
  'mutated prompt': 'How can I turn this complex problem into a series of simple, bite-sized tasks?',
  'details': "The mutation involves rephrasing the original prompt to make it more visual and relatable by using the terms 'complex problem' and 'bite-sized tasks'. This encourages the user to think of the problem as something that can be easily digested in smaller pieces."},
 'child prompt 2': {'original prompt': 'What are the key assumptions and underlying causes contributing to this problem?',
  'mutated prompt': 'What hidden beliefs and root factors are fueling this issue?',
  'details': "The mutation replaces specific keywords with related but unexpected terms: 'key assumptions' becomes 'hidden beliefs', and 'underlying causes' becomes 'root factors'. This encourages a fresh perspective on identifying the foundational elements of the problem."},
 'child

### Stage 5: Evaluation

Use swarm intelligence to perform evaluation using majority voting to select top mutated prompt for each offspring.

In [24]:
class ReviewPrincipals:
    def __init__(self, core_principles: list):
        self.core_principles = core_principles
    
    def add_principle(self, principle: str):
        """
        Adds a principle to the core principles list.
        
        :param principle: The principle to be added.
        """
        self.core_principles.append(principle)
        
    def __str__(self):
        """
        Returns a string representation of the core principles, each principle is listed on a new line with a preceding dash.
        
        Example:
        - principle 1
        - principle 2
        ...
        """
        return "\n".join([f"- {principle}" for principle in self.core_principles])


class ReviewAgent:
    """
    This class represents a voting agent that reviews a selection of prompts and votes for the best based on its persona and core principles.
    """
    def __init__(self, persona_title: str, core_principles: ReviewPrincipals, model: str='gpt-4o', temperature: float=1.0):
        """
        Initializes the PromptReviewAgent with a persona title, core principles, model, and temperature.
        
        :param persona_title: The title of the persona for the agent.
        :param core_principles: The core principles of the agent.
        :param model: The model to be used by the agent. Default is 'gpt-4o'.
        :param temperature: The temperature to be used by the agent. Default is 1.0.
        """
        self.persona_title = persona_title
        self.core_principles = str(core_principles)
        self.llm = ChatOpenAI(temperature=temperature, model=model)
    
    async def review(self, prompts_to_review: dict):
        """
        Reviews multiple sets of prompts and returns the best from each set.
        
        :param prompts_to_review: The prompts to be reviewed.
        :return: A results dict.
        """
        instruction = f"""You are a senior {self.persona_title}. 
        You have been asked to review the following mutated prompts: {prompts_to_review}.
        You should consider the prompts in light of your core principles, providing positive and negative feedback for each prompt.
        You should also score the prompts based on how well they align with your core principles. The score should be an integer between 0 and 100.
        
        Your core principles are:
        {self.core_principles}

        The output must be json code with the following format:
        {{
            "mutation prompt 1": {{
                    "orignal prompt": "orignal prompt",
                    "mutated prompt": "mutated prompt",
                    "positives": "positives of the prompt",
                    "negatives": "negatives of the prompt",
                    "score": "score of the prompt",
                }},
                ...
        }}
        """

        model_output = await self.llm.ainvoke(instruction)
        json_output = get_json(model_output)

        if json_output is None:
            json_output = model_output.content

        return json.loads(json_output)

In [25]:
import asyncio

class SwarmEvaluation():
    """
    Use swarm intelligence to perfrom evaluation using majority voting to select top mutated prompt for each offspring.
    Each agent in the swarm is an expert on a single aspect of the requirements for the prompt.
    """
    
    def __init__(self, prompts: dict):
        """
        Initializes the SwarmEvaluation
        """
        self.prompts = prompts
        self.reviewers = {}
        self.review_results = {}

    def register_reviewer(self, reviewer: ReviewAgent):
        """
        Register a voter for the majority vote process.
        """
        self.reviewers[reviewer.persona_title] = reviewer

    async def review(self):
        """
        Executes the voting process by calling each voter to vote on the prompt in parallel. 
        Concatenates the votes and returns the result as a dict with the voter persona titles as the keys.
        """
        tasks = []
        for persona_title, reviewer in self.reviewers.items():
            task = asyncio.create_task(reviewer.review(self.prompts))
            tasks.append(task)
        
        results = await asyncio.gather(*tasks)
        
        for i, persona_title in enumerate(self.reviewers.keys()):
            self.review_results[persona_title] = results[i]
        
        return self.review_results  


In [26]:
swarm_evaluation = SwarmEvaluation(mutated_offspring)

In [27]:
code_quality_principals = ReviewPrincipals([
    "Always write clean code",
    "Always write tests",
    "Always write documentation"
    ])
code_quality_reviewer = ReviewAgent("software engineer", code_quality_principals)

architecture_quality_principals = ReviewPrincipals([
    "Always design for scalability",
    "Always design for maintainability",
    "Always design for performance"
    ])
architecture_quality_reviewer = ReviewAgent("software architect", architecture_quality_principals)

swarm_evaluation.register_reviewer(code_quality_reviewer)
swarm_evaluation.register_reviewer(architecture_quality_reviewer)

In [28]:
reviews = await swarm_evaluation.review()
reviews

{'software engineer': {'mutation prompt 1': {'original prompt': 'How can I simplify the problem by breaking it down into smaller, more manageable parts?',
   'mutated prompt': 'Imagine you are a sculptor chiseling a large block of marble. How would you chip away at this problem, piece by piece, to reveal a simpler, more manageable form?',
   'positives': 'The metaphorical scenario makes the prompt more engaging and visual, which can help in creative thinking and problem-solving.',
   'negatives': 'The metaphor may not resonate with everyone and could be seen as too abstract. Some might find it challenging to translate the analogy back to concrete steps.',
   'score': 70},
  'mutation prompt 2': {'original prompt': 'What are the key assumptions and underlying causes contributing to this problem?',
   'mutated prompt': 'Pretend you are a detective solving a mystery. What are the critical assumptions and hidden factors that led to this problem?',
   'positives': 'The role-playing element 

### Stage 6: New Generation

Generation an new generation of prompts inspired by the results of the previous iteration

In [43]:
class NewGenerationAgent:

    def __init__(self, n: int, reviews: dict):
        self.reviews = reviews
        self.n = n

    def generate(self):

        # Load openai language model
        llm = ChatOpenAI(
            api_key=openai_api_key,
            temperature=0.5,
            model='gpt-4o'
        )

        # Agent instruction
        instruction = f"""
        You are an expert thinker and problem solver. Generate a new generation of {self.n} prompts based on the following review results: {self.reviews}.
        Use the feedback from the reviewers generate improved prompts each with a unique thinking style. 
        Incorporate the positive feedback and address the negative feedback to ensure the new prompts are an improvement over the original ones.

        The output must be json code containing the new generation of prompts with the following format:
        {{
            "prompt 1": "prompt content",
            "prompt 2": "prompt content",
            ...
        }}
        """

        # Generate new generation of prompts
        model_output = llm.invoke(instruction)
        json_output = get_json(model_output)

        if json_output is None:
            json_output = model_output.content

        return json.loads(json_output)

In [60]:
new_generation = NewGeneration(10, reviews)
next_generation = new_generation.generate()

In [61]:
next_generation

{'prompt 1': "Write a recursive function to calculate the factorial of a number. Ensure to handle edge cases such as negative numbers, zero, and non-integer inputs with appropriate error handling. Additionally, provide comprehensive test cases and document the function's usage and limitations.",
 'prompt 2': 'Create an iterative function to compute the factorial of a number. Explore methods to parallelize the computation to improve performance for large input values. Include detailed tests and documentation to validate and explain your approach.',
 'prompt 3': 'Design a dynamic programming-based function with memoization to find the factorial of a number. Compare its execution time with recursive and iterative methods. Ensure to write tests and provide documentation to validate and explain the different approaches and their trade-offs.',
 'prompt 4': "Develop a recursive function to determine the factorial of a number, handling edge cases like negative values, zero, and non-integer inp

### Stage 7: Best Prompt Selection

Select the best prompt from the final pool of prompts

In [51]:
class PromptSelectionAgent:

    def __init__(self, prompts: dict):
        self.prompts = prompts

    def select(self):

        # Load openai language model
        llm = ChatOpenAI(
            api_key=openai_api_key,
            temperature=0.5,
            model='gpt-4o'
        )

        # Agent instruction
        instruction = f"""
        You are an expert prompt engineer. Select the best prompt from the following list of prompts: {self.prompts}.
        Consider the task at hand and the end user (an LLM) of the prompt.
        Justify why you think the selected prompt is the best choice.

        The output must be json code containing the selected prompt with the following format:
        {{
            "selected prompt": "prompt content",
            "justification": "justification"
        }}
        """

        # Generate new generation of prompts
        model_output = llm.invoke(instruction)
        json_output = get_json(model_output)

        if json_output is None:
            json_output = model_output.content

        return json.loads(json_output)

In [50]:
best_prompt_selection = PromptSelectionAgent(next_generation)
best_prompt = best_prompt_selection.select()

NameError: name 'next_generation' is not defined

In [ ]:
best_prompt

### Entire Cyle

Cycle through the algorithm, generating additional prompts using the feedback from the previous cycle to guide generation and speed up convergence.

In [52]:
class EvolutionaryCycle:
    """
    Perform an evolutionary cycle to generate new ideas from previous cycle results.
    """
    
    def __init__(self, task: str, thinking_styles: dict, mutator_prompts: list, pool_size: int, no_pairs: int):
        self.task = task
        self.thinking_styles = thinking_styles
        self.mutator_prompts = mutator_prompts
        self.pool_size = pool_size
        self.no_pairs = no_pairs

    async def run(self):
        """
        Run the evolutionary cycle.
        """
        thinking_styles = ThinkingStylesAgent(self.pool_size, self.task)

        prompts = thinking_styles.generate()
        cycles = 0
        reviews = {}

        while True:
            
            if cycles == 5:
                break

            breeder_agent = BreederAgent(prompts, self.no_pairs)
            offspring = breeder_agent.generate()

            mutation_agent = MutationAgent(offspring, self.mutator_prompts)
            mutated_offspring = mutation_agent.generate()

            swarm_evaluation = SwarmEvaluation(mutated_offspring)

            code_quality_principals = ReviewPrincipals([
                "Always write clean code",
                "Always write tests",
                "Always write documentation"
                ])
            code_quality_reviewer = ReviewAgent("software engineer", code_quality_principals)

            architecture_quality_principals = ReviewPrincipals([
                "Always design for scalability",
                "Always design for maintainability",
                "Always design for performance"
                ])
            architecture_quality_reviewer = ReviewAgent("software architect", architecture_quality_principals)

            prompt_quality_principals = ReviewPrincipals([
                "Always write clear and concise prompts",
                "Always provide examples and context in prompts",
                "Always ensure prompts are actionable and specific"
                ])
            prompt_quality_reviewer = ReviewAgent("prompt engineer", prompt_quality_principals)
            

            swarm_evaluation.register_reviewer(code_quality_reviewer)
            swarm_evaluation.register_reviewer(architecture_quality_reviewer)
            swarm_evaluation.register_reviewer(prompt_quality_reviewer)

            reviews = await swarm_evaluation.review()

            new_generation = NewGenerationAgent(self.pool_size, reviews)
            prompts = new_generation.generate()

            cycles += 1
            print(f"Cycles Complete: {cycles}")
            
        best_prompt_selection = PromptSelectionAgent(prompts)
        best_prompt = best_prompt_selection.select()

        return best_prompt, cycles

In [53]:
task = "write a function that outputs the fibonacci sequence up to a given number"

In [54]:
evolutionary_cycle = EvolutionaryCycle(task, thinking_styles, mutator_prompts, 10, 6)

In [55]:
# import cProfile
# import pstats
# import io

# pr = cProfile.Profile()
# pr.enable()

best_prompt, cycles = await evolutionary_cycle.run()

# pr.disable()
# s = io.StringIO()
# sortby = 'cumtime'
# ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
# ps.print_stats()
# print(s.getvalue())

Cycles Complete: 1
Cycles Complete: 2
Cycles Complete: 3
Cycles Complete: 4
Cycles Complete: 5


In [56]:
best_prompt, cycles

NameError: name 'best_prompt' is not defined